In [1]:
from pathlib import Path
import os.path
import os
import numpy as np
import re
import pandas as pd
import shutil 
import networkx as nx

import powerlaw
#from networkx.algorithms import community
import networkx.algorithms.community as nxcom
import community as community_louvain
import matplotlib.pyplot as plt
import time
import collections

In [2]:
soft_path = 'D:/mm_thesis/Software_graphs/D_graph/'

softwareList = os.listdir(soft_path)
soft_files = [(os.path.join(soft_path,infile)+"/") for infile in softwareList]
#csv_path = ("D:/mm_thesis/SIG_csv/")
csv_path = 'D:/mm_thesis/Software_graphs/D_graph/D_sys/'
csv_path_l = os.listdir(csv_path)
csv_files1 = [(os.path.join(csv_path,infile)+"/") for infile in csv_path_l if os.path.isdir(os.path.join(csv_path,infile))]

csv_files = [i for i in csv_files1 if i.find("Time") == -1]

In [3]:
csv_files

['D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/']

In [7]:
net_csv = csv_files[5:]

In [5]:

for soft in csv_files:
#for soft in net_csv:
    print('soft', soft)
    soft_all = soft[:-1]
    soft_st = soft_all.rfind('/')
    soft_name = soft_all[soft_st+1 :]
    name_save = soft_name +'_Inh_m.csv'
    c_path = os.path.join(soft,name_save)
    print(c_path)
    soft_list = os.listdir(soft)
    
    
    soft_files = [(os.path.join(soft,infile )+"/")for infile in soft_list if os.path.isdir(os.path.join(soft,infile))]

soft D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/
D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/Jena_Inh_m.csv


In [6]:
print(soft_files)

['D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/CDot_time/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/collDot_time/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/InhDot_time/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.10.0/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.10.1/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.11.0/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.11.1/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.11.2/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.12.0/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.13.0/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.7.1/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.7.2/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.7.3/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.7.4/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-3.0.0/', 'D:/mm_thesis/Software_graphs/D_graph/D_sys/J

In [13]:
column_names = ['Version','t_stmp','no_nodes', 'no_edges','avg_degree',
                'no_comm','avg_shortest_path','sub_density',
                'avg_clust','SCC','WCC', 'LWC_nodes','LWC_edgeds','inCount','outCount' ]

for soft in csv_files:

    print('soft', soft)
    soft_all = soft[:-1]
    soft_st = soft_all.rfind('/')
    soft_name = soft_all[soft_st+1 :]
    soft_list = os.listdir(soft)
    
    soft_files = [(os.path.join(soft,infile )+"/")for infile in soft_list if os.path.isdir(os.path.join(soft,infile))]
#     soft_dir = [(os.path.join(soft,infile )+"/")for infile in soft_list if os.path.isdir(os.path.join(soft,infile))]
#     print(soft_files)
    net_list = []
    for ver in soft_files:
#         print(f_d)
#         print('ver', ver)
        f1 = os.listdir(ver)
#         f2 = [ os.path.join(ver,l) for l in f1 if (os.path.isdir(os.path.join(ver,l)) and l.find("CDot_csv")!= -1)  ]
        f2 = [ os.path.join(ver,l) for l in f1 if (os.path.isdir(os.path.join(ver,l)) and l.find("CollDot_csv")!= -1)  ]
#         f2 = [ os.path.join(ver,l) for l in f1 if (os.path.isdir(os.path.join(ver,l)) and l.find("InhDot_csv")!= -1)  ]
        
        
        for  f_d in f2:
#             print(f_d)
            net_dir = os.listdir(f_d)
    
            net_f = [d for d in net_dir if d.find('p_edges')!= -1]
#             print('net_f    ', net_f )
            net_path =  f_d+"/" +net_f[0] 
            net_files = net_path
#             print('net_path :' ,net_path)
            net_list.append(net_files)
    print('net_list', len(net_list))
#     for i in net_list:
#         print(i)
#     print('intialize info list for soft', soft)
    info_list = []
    x = 0
    
    for i in net_list:
#        
        e1 =  pd.read_csv(i)
        i_name   = e1["Version"].tolist()[0]
        print('i_name  ',i_name)
        ts = e1['t_stmp'].tolist()[0]
        #         print(ts)
#         x = i_name
        ts = e1.at[0,'t_stmp']
        print(ts)
        print('i   :',i)
#   
        sub = nx.from_pandas_edgelist(e1, source = 'source', target='target',edge_attr=True, create_using=nx.MultiDiGraph())
# #                                       nx.DiGraph())
        
       
        
        
        no_nodes, no_edges,avg_deg,no_comm, gc_avg_shortest_path,sub_density,avg_clust,SCC,WCC,LWC_nodes,LWC_edges,incount, outcount =get_info_path(sub)
         
        info_list.append([i_name,ts,no_nodes,no_edges, avg_deg,no_comm,
                          gc_avg_shortest_path,sub_density,avg_clust,SCC,WCC,LWC_nodes,LWC_edges,incount,outcount])
    
    print("putting info ", soft)
    
    
    nt_d = pd.DataFrame(info_list)
#     print(nt_d.head())
    nt_d.columns = column_names
#     name_save = soft_name +'_cdot_m.csv'
    name_save = soft_name +'_coll_m.csv'
#     name_save = soft_name +'_Inh_m.csv'
    C_path = os.path.join(soft,name_save)

    print(C_path)
    nt_d.to_csv(C_path, encoding='utf-8', index=False)
         
    
            


    #     C_path = soft+'info_cdot_m.csv'
#     C_path =soft +'info_colldot_m.csv'
#     C_path =soft +'info_Inhdot_m.csv'

soft D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/
net_list 32
i_name   jena-jena-2.10.0
1361318400
i   : D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.10.0/CollDot_csv/Ver_stmp_edges.csv
i_name   jena-jena-2.10.1
1368230400
i   : D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.10.1/CollDot_csv/Ver_stmp_edges.csv
i_name   jena-jena-2.11.0
1378944000
i   : D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.11.0/CollDot_csv/Ver_stmp_edges.csv
i_name   jena-jena-2.11.1
1390003200
i   : D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.11.1/CollDot_csv/Ver_stmp_edges.csv
i_name   jena-jena-2.11.2
1401667200
i   : D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.11.2/CollDot_csv/Ver_stmp_edges.csv
i_name   jena-jena-2.12.0
1406332800
i   : D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.12.0/CollDot_csv/Ver_stmp_edges.csv
i_name   jena-jena-2.13.0
1431043200
i   : D:/mm_thesis/Software_graphs/D_graph/D_sys/Jena/jena-2.13.0/CollDot_csv/Ver_stmp_edges.

In [10]:
def get_info_path (sub):
    
#      S I Z E   I N F O 
    no_edges = sub.number_of_edges()
    no_nodes = sub.number_of_nodes()
    #nnodes = sub.number_of_nodes()
#     sub_in = sub.in_degree()
#     sub_out = sub.out_degree()
#     avg_deg_in = sum(d for n, d in sub.in_degree()) / float(no_nodes)

#     avg_deg_out = sum(d for n, d in sub.out_degree()) / float(no_nodes)
    nodes_sub = [n for n in sub]
    avg_deg_in = 0
    avg_deg_out = 0
    for i in nodes_sub: 
        avg_deg_in =  avg_deg_in +  sub.in_degree(i)
        avg_deg_out =  avg_deg_out +  sub.out_degree(i)
        
    avg_deg = avg_deg_in / float(no_nodes) 
#     avg_deg_out = avg_deg_out / float(no_nodes)

####  D E G R E E   D I S T R I B U T I O N 

    in_degrees = sub.in_degree() # dictionary node:degree
    in_values = [val for (node, val) in sorted(in_degrees(), key=lambda pair: pair[0], reverse = True)]
    inCount = collections.Counter(in_values)
    out_degrees = sub.out_degree()
#     print('out_degrees ==>', out_degrees)
    out_values = [val for (node, val) in sorted(out_degrees(), key=lambda pair: pair[0],reverse= True)]
    outCount = collections.Counter(out_values)
    
#  G E T   T H E  C O N N E C T E D  C O M P O N E N T S     
    
    SCC = nx.number_strongly_connected_components(sub)
    WCC = nx.number_weakly_connected_components(sub)     # To get the shortest patg 

    largest_cc = max(nx.weakly_connected_components(sub), key=len)
    s = sub.subgraph(largest_cc)
    
    LWC_edges = s.number_of_edges()
    LWC_nodes = s.number_of_nodes()
        
    
    
    sub_density = nx.density(sub)
#     print("DENSITY :", sub_density)
    sub_un = sub.to_undirected()
    
#     part = community_louvain.best_partition(sub_un)
#     print( max(part.values())
    
#     reslt = nxcom.girvan_newman(sub_un)
#     print(type(reslt))
#     print(reslt)
#     comm = next(reslt)
#     print(comm)
#     try:
        
# #         comm = next(reslt)
#         no_comm = len(comm)
#         print(no_comm)
#     except Exception as e:
#         print(e)
#         avg_clust = 0
    communities = sorted(nxcom.greedy_modularity_communities(sub_un), key=len, reverse=True)
    no_comm = len(communities)
#     print("comm", no_comm)
#     print(communities[0])
    
    di_sub = nx.DiGraph(sub)
    try:
        avg_clust = nx.average_clustering (di_sub)
#         print('cluster' , avg_clust)
#     except Exception as e:
#         print(e)
    except Exception:
        avg_clust = 0
        print('err clust')
        pass 
    
#     fit = powerlaw.Fit(s.nodes)
#     p_law = fit.power_law.alpha
#     print("power_law", p_law)
    gc_avg_shortest_path = 0
    
    try:
#         st1 = time.time()
        gc_avg_shortest_path = nx.average_shortest_path_length(s)
#         end1 = time.time()
#         print('time for path', end1- st1)
        
    except Exception as e:
        print(e)
        
    except Exception:
        print("no wCC")
        gc_avg_shortest_path = 0
        
        pass
        
   
    return (no_nodes,no_edges,avg_deg, no_comm, gc_avg_shortest_path,sub_density,avg_clust,SCC,WCC,
            LWC_nodes,LWC_edges,in_values,out_values)